In [8]:
import pandas as pd
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn import mixture
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from statsmodels.tools import add_constant
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ConstantKernel as C
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, RationalQuadratic, DotProduct
import statsmodels.api as sm
import random
from matplotlib import pyplot as plt
import os
import shapely

import time
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
# from collections import Counter
import pickle

%matplotlib inline
sns.set(style="ticks")

want to see what cell sensor is in
use cell info data to determine land usage

In [84]:
AQ_width = AQ.shape[1]
LU_width = LU.shape[1]
col_names = ['Cropland', 'Golf Course',
       'Saltwater Sandy Beach', 'cemetary', 'commercial', 'crop_land',
       'forest', 'high_density_residential', 'industrial',
       'low_density_residential', 'marina', 'medium_density_residential',
       'mining', 'open_land', 'recreational', 'transitional', 'transportation',
       'urban_public/institutional', 'waste', 'water', 'wetland']

In [25]:
# generate grid vertices/ticks

x = [ -71.20197, -70.96679]
y = [42.291441, 42.420578]
x_cell = 50
y_cell = 50

x_min = -71.20197
x_max = -70.96679
y_min = 42.291441
y_max = 42.420578

# create ticks
x_s = np.linspace(x_min, x_max, x_cell + 1)
y_s = np.linspace(y_min, y_max, y_cell + 1)

In [26]:
# given lat/lon spot
# return indicies for lat/lon grid box bounds
# where bounds are:
# y_s[lat_index - 1], y_s[lat_index]
# x_s[lon_index - 1], x_s[lon_index]
def find_gridbox(lat, long):
    # locate correct grid box bounds
    lat_index = 0
    lon_index = 0
    for i in range(0, 50):
        if x_s[i] < long:
            lon_index = i
        if y_s[i] > lat:
            lon_index = i
    return (lat_index, lon_index)

In [27]:
# function to parse string from "Grids" column in boston_preds.csv
# returns list of grid vertices as float64 tuples
def parse_str(str_edit):
    str_edit = str_edit.replace('[', '')
    str_edit = str_edit.replace('(', '')
    str_edit = str_edit.replace(']', '')
    str_edit = str_edit.replace(')', '')
    str_edit = str_edit.replace(',', '')
    list = str_edit.split(' ')
    return [(np.float64(list[0]), np.float64(list[1])), (np.float64(list[2]), np.float64(list[3])),
            (np.float64(list[4]), np.float64(list[5])), (np.float64(list[6]), np.float64(list[7]))]

In [28]:
def in_cell(lat, lon, cell):
    cell_points = parse_str(cell)
    lat_bool = lat > cell_points[0][0] and lat < cell_points[2][0]
    lon_bool = lon > cell_points[0][1] and lon < cell_points[2][1]
    if lat_bool and lon_bool:
        return True
    else:
        return False

In [64]:
def add_columns(df):
    
    df['Cropland'] = 0
    df['Golf Course'] = 0
    df['Saltwater Sandy Beach'] = 0
    df['cemetary'] = 0
    df['commercial'] = 0
    df['crop_land'] = 0
    df['forest'] = 0
    df['high_density_residential'] = 0
    df['industrial'] = 0
    df['low_density_residential'] = 0
    df['marina'] = 0
    df['medium_density_residential'] = 0
    df['mining'] = 0
    df['open_land'] = 0
    df['recreational'] = 0
    df['transitional'] = 0
    df['transportation'] = 0
    df['urban_public/institutional'] = 0
    df['waste'] = 0
    df['water'] = 0
    df['wetland'] = 0
    
    return df

In [67]:
def update_proportions(AQ_df, LU_df, AQ_index, LU_index):
    
    for col in range(0, LU_width - 1):
        AQ_df.set_value(AQ_index, col + AQ_width, LU_df.loc[LU_index][col + 1])

    return AQ_df

In [87]:
def update_proportions2(AQ_df, LU_df, AQ_index, LU_index):
    
    for col in col_names:
        AQ_df.set_value(AQ_index, col, LU_df.loc[LU_index][col])

    return AQ_df

In [85]:
AQ = pd.read_csv('appendedAQ.csv')
lat = AQ['Latitude']
lon = AQ['Longitude']

AQ['lat_lon'] = list(zip(lat, lon))
pts = AQ['lat_lon']

LU = pd.read_csv('boston_site_LU.csv')
sites = LU['Site']

AQ = add_columns(AQ)

In [88]:
for pt in range(len(AQ['lat_lon'])):
    for cell in range(len(sites)):
        if in_cell(pts[pt][1], pts[pt][0], LU['Site'][cell]):
            # add cell info to appendedAQ.csv file
            AQ = update_proportions2(AQ, LU, pt, cell)

In [89]:
AQ.columns

Index(['Unnamed: 0', '\t\tState Code', 'County Code', 'Tribal Code', 'Site ID',
       'Support Agency Code', 'Location Address', 'City Code', 'Postal Code',
       'Local ID', 'Local Name', 'Urban Area Code', 'AQCR Code', 'Land Use ID',
       'Location Setting ID', 'Site Established Date', 'Latitude', 'Longitude',
       'Horizontal Method Code', 'Horizontal Datum ID', 'Parameter Code',
       'Parameter Name', 'Substance Occurrence Code', 'Duration Code',
       'Method ID', 'Measure Unit Code', 'Sample Collection Start Date',
       'Sample Collection Start Time', 'Measure Value', 'Measure Unit',
       'Null Data Code', 'Qualifier Code', 'Data Validity Code', 'lat_lon',
       'Cropland', 'Golf Course', 'Saltwater Sandy Beach', 'cemetary',
       'commercial', 'crop_land', 'forest', 'high_density_residential',
       'industrial', 'low_density_residential', 'marina',
       'medium_density_residential', 'mining', 'open_land', 'recreational',
       'transitional', 'transportation'

In [90]:
AQ.head

<bound method NDFrame.head of      Unnamed: 0 \t\tState Code County Code  Tribal Code     Site ID  \
0             0         \t\tMA        '025          NaN  '250250042   
1             1     \t\t\t\tMA        '025          NaN  '250250042   
2             2     \t\t\t\tMA        '025          NaN  '250250042   
3             3     \t\t\t\tMA        '025          NaN  '250250042   
4             4     \t\t\t\tMA        '025          NaN  '250250042   
5             5     \t\t\t\tMA        '025          NaN  '250250042   
6             6     \t\t\t\tMA        '025          NaN  '250250042   
7             7     \t\t\t\tMA        '025          NaN  '250250042   
8             8     \t\t\t\tMA        '025          NaN  '250250042   
9             9     \t\t\t\tMA        '025          NaN  '250250042   
10           10     \t\t\t\tMA        '025          NaN  '250250042   
11           11     \t\t\t\tMA        '025          NaN  '250250042   
12           12     \t\t\t\tMA        '025     